In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame

In [4]:
PARTS_MASTER = pd.read_pickle('data/temp01/PARTS_MASTER.pkl')
SALES_ORDERS = pd.read_pickle('data/temp01/SALES_ORDERS.pkl')
WO_BOM = pd.read_pickle('data/temp01/WO_BOM.pkl')
WO_HEADER = pd.read_pickle('data/temp01/WO_HEADER.pkl')
PURCHASING = pd.read_pickle('data/temp01/PURCHASING.pkl')
STOCK = pd.read_pickle('data/temp01/STOCK.pkl')

#sum all tables with Quantities per PN / PNM couple
SALES_ORDERS_VOL = pd.pivot_table(SALES_ORDERS, index=['PNM','PN'],values = 'QTYORDERED', aggfunc = np.sum)
SALES_ORDERS_VOL.reset_index(inplace = True)
SALES_ORDERS_VOL.rename(columns={'QTYORDERED':'QTY'},inplace = True)

WO_BOM_VOL = pd.pivot_table(WO_BOM, index=['PNM','PN'],values = 'QTY_REQ', aggfunc = np.sum)
WO_BOM_VOL.reset_index(inplace = True)
WO_BOM_VOL.rename(columns={'QTY_REQ':'QTY'},inplace = True)

WO_HEADER_VOL  = pd.pivot_table(WO_HEADER, index=['PNM','PN'], values = 'WONO', aggfunc=lambda x: len(x.unique()))
WO_HEADER_VOL.reset_index(inplace = True)
WO_HEADER_VOL.rename(columns = {'WONO':'QTY'},inplace=True)

PURCHASING_VOL = pd.pivot_table(PURCHASING, index=['PNM','PN'], values = 'QTYRECEIVED', aggfunc= np.sum)
PURCHASING_VOL.reset_index(inplace = True)
PURCHASING_VOL.rename(columns={'QTYRECEIVED':'QTY'},inplace = True)

STOCK_VOL = pd.pivot_table(STOCK, index=['PNM','PN'], values = 'QTY', aggfunc= np.sum)
STOCK_VOL.reset_index(inplace = True)

#Sum all usage in one column in Parts_master
ALL_VOLS = DataFrame()
ALL_VOLS = ALL_VOLS.append([SALES_ORDERS_VOL, WO_BOM_VOL],ignore_index=True,sort=False)
ALL_VOLS = pd.pivot_table(ALL_VOLS,index=['PNM','PN'],aggfunc= np.sum, values=['QTY'])
ALL_VOLS.reset_index(inplace=True)
ALL_VOLS = ALL_VOLS.merge(PARTS_MASTER[['PN','PNM']],left_on='PNM',right_on='PNM',how='left')
ALL_VOLS.rename(columns={'PN_x':'PN','PN_y':'GOOD_PN'},inplace = True)
ALL_VOLS = pd.pivot_table(ALL_VOLS,index=['PNM','GOOD_PN'],aggfunc= np.sum, values=['QTY'])
ALL_VOLS.reset_index(inplace=True)
ALL_VOLS.rename(columns={'GOOD_PN':'PN'},inplace = True)

#bring back QTY on parts master
PARTS_MASTER = PARTS_MASTER.merge(ALL_VOLS,left_on = ['PNM','PN'],right_on = ['PNM','PN'],how='left')
PARTS_MASTER['QTY'] = PARTS_MASTER['QTY'].fillna(0)

#add score points if description is not "conversion", if STC is not a weird one, if List_price exists
PARTS_MASTER['SCORE'] = 0
PARTS_MASTER['SCORE'] = np.where(PARTS_MASTER['DESCRIPTION'] !='Conversion', PARTS_MASTER['SCORE'] + 3, PARTS_MASTER['SCORE'])
PARTS_MASTER['SCORE'] = np.where(PARTS_MASTER['STC'] !='_', PARTS_MASTER['SCORE'] + 1, PARTS_MASTER['SCORE'])
PARTS_MASTER['SCORE'] = np.where(PARTS_MASTER['CLASS'] !='_', PARTS_MASTER['SCORE'] + 1, PARTS_MASTER['SCORE'])
PARTS_MASTER['SCORE'] = np.where(PARTS_MASTER['CLASS'] !='NE', PARTS_MASTER['SCORE'] + 1, PARTS_MASTER['SCORE'])
PARTS_MASTER['SCORE'] = np.where(~PARTS_MASTER['PRICEDATE'].isnull(), PARTS_MASTER['SCORE'] + 1, PARTS_MASTER['SCORE'])

PARTS_MASTER['TOTAL_SCORE'] = PARTS_MASTER['SCORE'] + PARTS_MASTER['QTY']
PARTS_MASTER.sort_values(by=['PN','TOTAL_SCORE'],inplace = True,ascending = False)

#record best PNM for PNs
TEMP_BEST_PNM = PARTS_MASTER[['PN','PNM']].drop_duplicates(subset='PN',keep='first').rename(columns={'PNM':'BEST_PNM'})

#bring back best PNM into parts_master
PARTS_MASTER = PARTS_MASTER.merge(
    TEMP_BEST_PNM,
    left_on = 'PN',right_on='PN',how='left')

#create table with best PNM
BEST_PNM = PARTS_MASTER[['PNM','BEST_PNM']]

#save
PARTS_MASTER.to_pickle('data/temp02/PARTS_MASTER.pkl')
BEST_PNM.to_pickle('data/temp02/BEST_PNM.pkl')